In [1]:
#GATv2 TEST 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [13]:
N = 2708
in_features = 1433
out_features = 64
alpha = 0.2
adj = torch.tensor(np.ones((N,N)), dtype=torch.float)
h = torch.tensor(np.random.randn(N,in_features), dtype=torch.float)

In [19]:

W = nn.Parameter(torch.empty(size=(in_features, 2*out_features)))
nn.init.xavier_uniform_(W.data, gain=1.414)
a = nn.Parameter(torch.empty(size=(out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)

leakyrelu = nn.LeakyReLU(alpha)


#Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
Wh1 = torch.matmul(h,W[:, :out_features]) #N, F @ F , F' --> N, F'
Wh2 = torch.matmul(h,W[:, out_features:])
e_0 = Wh1 + Wh2 
e_1 = leakyrelu(e_0)
e = torch.matmul(e_1, a) #N=2708, F'=64 @ F' --> N
      
#e = self._prepare_attentional_mechanism_input(Wh)

zero_vec = -9e15*torch.ones_like(e)
attention = torch.where(adj > 0, e, zero_vec)
attention = F.softmax(attention, dim=1)
h_prime = torch.matmul(attention, Wh2)



In [24]:
#OTHER TEST 
W = nn.Parameter(torch.empty(size=(2*in_features, out_features)))
nn.init.xavier_uniform_(W.data, gain=1.414)
a = nn.Parameter(torch.empty(size=(out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)

leakyrelu = nn.LeakyReLU(alpha)


#Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
Wh1 = torch.matmul(h,W[:in_features, :]) #N, F @ F , F' --> N, F'
Wh2 = torch.matmul(h,W[in_features:, :])
e_0 = Wh1 + Wh2 
e_1 = leakyrelu(e_0)
e = torch.matmul(e_1, a) #N=2708, F'=64 @ F' --> N
      
#e = self._prepare_attentional_mechanism_input(Wh)

zero_vec = -9e15*torch.ones_like(e)
attention = torch.where(adj > 0, e, zero_vec)
attention = F.softmax(attention, dim=1)
h_prime = torch.matmul(attention, Wh2)

In [25]:
print(Wh1.shape)
print(Wh2.shape)
print(e_0.shape)
print(e_1.shape)
print(a.shape)
h_prime.shape

torch.Size([2708, 64])
torch.Size([2708, 64])
torch.Size([2708, 64])
torch.Size([2708, 64])
torch.Size([64, 1])


torch.Size([2708, 64])